In [1]:
import os


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist


In [2]:
# Use Pandas to load dataset from csv file
import pandas as pd

# HYPERPARAMETERS
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001


In [4]:
# from google.colab import drive

# drive.mount("/content/gdrive")

ModuleNotFoundError: No module named 'google.colab'

In [4]:
datapath = "/content/gdrive/MyDrive/Deep Learning/Jupyter Notebook/datasets/archive.zip"
import zipfile
zip_ref = zipfile.ZipFile(datapath, 'r')
zip_ref.extractall("/content/sample_data")
zip_ref.close()

In [3]:
data_path = "/content/sample_data/"
train_df = pd.read_csv(data_path + "train.csv")
test_df = pd.read_csv(data_path + "test.csv")


FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/train.csv'

In [6]:
train_df.head(2)

,Image,first_num,second_num
0,0_00.png,0,0
1,100_00.png,0,0


import numpy as np
from google.colab import autoviz
df_4693396303902216436 = autoviz.get_df('df_4693396303902216436')

def categorical_histogram(df, colname, figsize=(2, 1.2), mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  _, ax = plt.subplots(figsize=figsize)
  bars = df[colname].value_counts()
  plt.barh(bars.index, bars.values, color=sns.palettes.mpl_palette(mpl_palette_name))
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(df_4693396303902216436, *['Image'], **{})
chart

In [7]:
train_images = data_path + "train_images/" + train_df.iloc[:, 0].values
test_images = data_path + "test_images/" + test_df.iloc[:, 0].values

In [8]:
train_images[0]

'/content/sample_data/train_images/0_00.png'

In [9]:
train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values

In [ ]:
train_labels

In [11]:
def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    # In older versions you need to set shape in order to avoid error
    # on newer (2.3.0+) the following 3 lines can safely be removed
    # image.set_shape((64, 64, 1))
    # label[0].set_shape([])
    # label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels


In [12]:
read_image(train_images[0],train_labels[0])

(<tf.Tensor: shape=(64, 64, 1), dtype=float32, numpy=
 array([[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        ...,
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]], dtype=float32)>,
 {'first_num': 0, 'second_num': 0})

In [13]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset.map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)


In [15]:
inputs = keras.Input(shape=(64, 64, 1))
x = layers.Conv2D(
    filters=32,
    kernel_size=3,
    padding="same",
    kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)

x = layers.Conv2D(64, 3, kernel_regularizer=regularizers.l2(WEIGHT_DECAY),)(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)

x = layers.Conv2D(
    64, 3, activation="relu", kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(x)
x = layers.Conv2D(128, 3, activation="relu")(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation="relu")(x)

output1 = layers.Dense(10, activation="softmax", name="first_num")(x)
output2 = layers.Dense(10, activation="softmax", name="second_num")(x)

model = keras.Model(inputs=inputs, outputs=[output1, output2])


In [16]:
model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)



In [17]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 32)   320         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 tf.nn.relu (TFOpLambda)        (None, 64, 64, 32)   0           ['batch_normalization[0][0]']

In [18]:
model.fit(train_dataset, epochs=5, verbose=1)
model.evaluate(test_dataset, verbose=1)

Epoch 1/5
1000/1000 [==============================] - 48s 33ms/step - loss: 1.8326 - first_num_loss: 0.8843 - second_num_loss: 0.8668 - first_num_accuracy: 0.6860 - second_num_accuracy: 0.6912
Epoch 2/5
1000/1000 [==============================] - 32s 31ms/step - loss: 0.5885 - first_num_loss: 0.2607 - second_num_loss: 0.2640 - first_num_accuracy: 0.9132 - second_num_accuracy: 0.9120
Epoch 3/5
1000/1000 [==============================] - 31s 31ms/step - loss: 0.3941 - first_num_loss: 0.1676 - second_num_loss: 0.1695 - first_num_accuracy: 0.9453 - second_num_accuracy: 0.9442
Epoch 4/5
1000/1000 [==============================] - 31s 31ms/step - loss: 0.3134 - first_num_loss: 0.1336 - second_num_loss: 0.1270 - first_num_accuracy: 0.9558 - second_num_accuracy: 0.9576
Epoch 5/5
313/313 [==============================] - 5s 14ms/step - loss: 1.0212 - first_num_loss: 0.3210 - second_num_loss: 0.6521 - first_num_accuracy: 0.9066 - second_num_accuracy: 0.8190


[1.021240472793579,
 0.32096707820892334,
 0.6521443128585815,
 0.9065999984741211,
 0.8190000057220459]